In [1]:
import pandas as pd
#from selenium import webdriver
from bs4 import BeautifulSoup
from pathlib import Path
import requests
import time
'''
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException 
'''
# Step 1: Create a session and load the page
'''
url1 = 'https://www.nba.com/stats/players/pullup?PerMode=Totals'
url2 = 'https://www.nba.com/stats/players/catch-shoot?PerMode=Totals'
url3 = 'https://www.nba.com/stats/players/defense-dash-lt6?PerMode=Totals'
url4 = 'https://www.nba.com/stats/teams/isolation?PerMode=Totals'
url5 = 'https://www.nba.com/stats/players/transition?PerMode=Totals&dir=D&sort=POSS'
'''
cs ='https://www.nba.com/stats/players/catch-shoot?PerMode=Totals'
pullup ='https://www.nba.com/stats/players/pullup?PerMode=Totals'

handoff = 'https://www.nba.com/stats/teams/hand-off?PerMode=Totals'
iso ='https://www.nba.com/stats/teams/isolation?PerMode=Totals'
trans ='https://www.nba.com/stats/teams/transition?PerMode=Totals'
bh='https://www.nba.com/stats/teams/ball-handler?PerMode=Totals'
rollman = 'https://www.nba.com/stats/teams/roll-man?PerMode=Totals'
postup = 'https://www.nba.com/stats/teams/playtype-post-up?PerMode=Totals'
spotup = 'https://www.nba.com/stats/teams/spot-up?PerMode=Totals'
cut = 'https://www.nba.com/stats/teams/cut?PerMode=Totals'
offscreen = 'https://www.nba.com/stats/teams/off-screen?PerMode=Totals'
putbacks = 'https://www.nba.com/stats/teams/putbacks?PerMode=Totals'
misc = 'https://www.nba.com/stats/teams/playtype-misc?PerMode=Totals'
#drives = 'https://www.nba.com/stats/teams/drives?PerMode=Totals'
#url_list = [url1,url2,url3,url4,url5]
url_list=[handoff,iso,trans,bh,rollman,postup,spotup,cut,offscreen,putbacks,misc]
#url_list =[url +'&SeasonType=Playoffs' for url in url_list]
url_list =[url +'&SeasonType=Regular+Season'for url in url_list]
#add tag specifying that the url list is for the regular seasonx
def check_exists_by_xpath(driver, xpath):
    try:
        driver.find_element(By.XPATH, xpath)
    except NoSuchElementException:
        return False
    return True
#url_list = [cs,pullup]
def get_tables(url_list):
    xpath = '//*[@id="__next"]/div[2]/div[2]/div[3]/section[2]/div/div[2]/div[3]/table'
    data = []
    options = webdriver.FirefoxOptions()
    driver = webdriver.Firefox(options=options)

    for url in url_list:
        print(url)
        
        driver.get(url)
        element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, xpath)))
        # Wait for the page to fully load
        driver.implicitly_wait(20)
        '''if check_exists_by_xpath(driver, "//a[contains(text(),'>')]/preceding-sibling::a[1]"):
            number_of_pages = int(driver.find_element(By.XPATH, "//a[contains(text(),'>')]/preceding-sibling::a[1]").text)
            print(number_of_pages)'''
        # Step 2: Parse HTML code and grab tables with Beautiful Soup
        soup = BeautifulSoup(driver.page_source, 'lxml')

        tables = soup.find_all('table')

        # Step 3: Read tables with Pandas read_html()
        dfs = pd.read_html(str(tables))

        df= dfs[-1]

        data.append(df)
    driver.close()
    return data
def get_playtypes(years,ps= False,p_or_t='t',defense= False):
    field_side = "offensive"
    if defense == True:
        field_side= "defensive"
    if p_or_t =='p':
        type ='P'
    else:
        type='T'
    stype = "Regular+Season"
    trail =''
    if ps == True:
        stype = "Playoffs"
        trail='/playoffs'
    headers = {
        "Host": "stats.nba.com",
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:72.0) Gecko/20100101 Firefox/72.0",
        "Accept": "application/json, text/plain, */*",
        "Accept-Language": "en-US,en;q=0.5",
        "Accept-Encoding": "gzip, deflate, br",

        "Connection": "keep-alive",
        "Referer": "https://stats.nba.com/"
    }
    playtypes = ['Transition','PRBallHandler','Spotup','Isolation','PRRollman','Postup','Misc','OffRebound','Cut','Handoff','OffScreen',]
    terms = ['trans.csv','bh.csv','spotup.csv','iso.csv','rollman.csv','postup.csv','misc.csv','putback.csv','cut.csv','handoff.csv','offscreen.csv']
    plays = ['tran','pr_ball','spot','iso','pr_roll','post','misc','oreb','cut','hand_off','off_screen']

    frames = []
    data_columns= ['TEAM', 'GP', 'POSS', 'FREQ%', 'PPP', 'PTS', 'FGM', 'FGA', 'FG%',
       'EFG%', 'FTFREQ%', 'TOVFREQ%', 'SFFREQ%', 'AND ONEFREQ%', 'SCOREFREQ%',
       'PERCENTILE']
    if type =="P":
        data_columns= ['PLAYER_NAME','PLAYER_ID','TEAM', 'GP', 'POSS', 'FREQ%',
                   'PPP', 'PTS', 'FGM', 'FGA', 'FG%','EFG%', 'FTFREQ%', 'TOVFREQ%', 'SFFREQ%', 'AND ONEFREQ%', 'SCOREFREQ%','PERCENTILE']
        
    for year in years:
        ssn = str(year)+'-'+str(year+1 - 2000)
        i = 0
        for play in playtypes:
            
            half1 = "https://stats.nba.com/stats/synergyplaytypes?LeagueID=00&PerMode=Totals&PlayType="+play+"&PlayerOrTeam="+type+"&SeasonType="+stype+"&SeasonYear="
            half2 = "&TypeGrouping="+field_side
            term = terms[i]
            url = (
                        
                        half1+ str(ssn)+half2
                        
                    )
            #print(url)
            json = requests.get(url,headers = headers).json()
            data = json["resultSets"][0]["rowSet"]
            
            columns = json["resultSets"][0]["headers"]

            df2 = pd.DataFrame.from_records(data, columns=columns)
                #df2.columns
          
            df2 = df2.rename(columns={'TEAM_NAME':'TEAM','POSS_PCT':'FREQ%','EFG_PCT':'EFG%','FG_PCT':'FG%',
                                          'TOV_POSS_PCT':'TOVFREQ%','PLUSONE_POSS_PCT':'AND ONEFREQ%','FT_POSS_PCT':'FTFREQ%','SCORE_POSS_PCT':'SCOREFREQ%','SF_POSS_PCT':'SFFREQ%'})
            for col in df2.columns:
                if '%' in col or 'PERC' in col:
                    df2[col]*=100
            #print(df2)
            path = str(year+1)+trail+'/playtype/'+term
          
            df2 = df2.round(2)
            
            df2 = df2[data_columns]
            
            #print(df2)
            if p_or_t.lower() =='t':
                if defense == False:
                    df2.to_csv(path,index = False)
                    
                
                df2['playtype'] = plays[i]
                df2['year']=year+1
                 
                frames.append(df2)
            else:
                #print(df2)
                df2['playtype'] = plays[i]
                df2['year']=year+1
                print(len(df2))
                frames.append(df2)
          
            i+=1
            
        if p_or_t.lower() =='p':
            data = pd.concat(frames)
            
            map_terms ={
            'PLAYER_NAME': 'Player',
          
            'TEAM': 'Team',
            'GP': 'GP',
            'POSS': 'Poss',
            'FREQ%': '% Time',
            'PPP': 'PPP',
            'PTS': 'Points',
            'FGM': 'FGM',
            'FGA': 'FGA',
            'FG%': 'FG%',
            'EFG%': 'aFG%',
            'FTFREQ%': '%FT',
            'TOVFREQ%': '%TO',
            'SFFREQ%': '%SF',
            'AND ONEFREQ%': 'AND ONEFREQ%',
            'SCOREFREQ%': '%Score',
            'PERCENTILE': 'Percentile',
            'playtype': 'playtype'
        }

            data.rename(columns=map_terms,inplace=True)
            return data
        else:
             data = pd.concat(frames)
             map_terms ={
    
            'PTS':'Points',
            'TEAM': 'full_name'}
             data.rename(columns=map_terms,inplace=True)
             team_dict= {'New York Knicks': 'NYK',
             'New Orleans Pelicans': 'NOP',
             'Oklahoma City Thunder': 'OKC',
             'Golden State Warriors': 'GSW',
             'Brooklyn Nets': 'BKN',
             'Houston Rockets': 'HOU',
             'Miami Heat': 'MIA',
             'Phoenix Suns': 'PHX',
             'Philadelphia 76ers': 'PHI',
             'Sacramento Kings': 'SAC',
             'Los Angeles Clippers': 'LAC',
             'Cleveland Cavaliers': 'CLE',
             'Detroit Pistons': 'DET',
             'Los Angeles Lakers': 'LAL',
             'Denver Nuggets': 'DEN',
             'Orlando Magic': 'ORL',
             'Indiana Pacers': 'IND',
             'Boston Celtics': 'BOS',
             'Toronto Raptors': 'TOR',
             'Charlotte Bobcats': 'CHA',
             'Washington Wizards': 'WAS',
             'Milwaukee Bucks': 'MIL',
             'Minnesota Timberwolves': 'MIN',
             'Atlanta Hawks': 'ATL',
             'Portland Trail Blazers': 'POR',
             'Memphis Grizzlies': 'MEM',
             'San Antonio Spurs': 'SAS',
             'Dallas Mavericks': 'DAL',
             'Utah Jazz': 'UTA',
             'Chicago Bulls': 'CHI',
             'Charlotte Hornets': 'CHA'}
            
             data['Team'] = data['full_name'].map(team_dict)
            
             return data
years = [2023]
playoffs = True
offense = get_playtypes(years,ps=playoffs)
def update_player_master(year,ps=False):
    trail = ''
    if ps == True:
        trail ='_p'
    years=[year-1]
    frames = get_playtypes(years,ps=ps,p_or_t='p')

    old =pd.read_csv('playtype'+trail+'.csv')
    old = old[old.year!=year]
    print(frames.columns)
    new = pd.concat([old,frames])
    new.to_csv('playtype'+trail+'.csv',index=False)
    return new
new = update_player_master(2024,ps=playoffs)
new2 = update_player_master(2024,ps=False)


defense = get_playtypes([2023],ps=True,defense=True)
def update_team_masters(year,offense,defense,ps=False):
    trail = ''
    if ps == True:
        trail='_p'
    offpath= 'teamplay'+trail+'.csv'
    defpath='teamplayd'+trail+'.csv'
    old_off =pd.read_csv(offpath)
    old_def =pd.read_csv(defpath)

    old_off = old_off[old_off.year!=year]

    new_off = pd.concat([old_off,offense])

    old_def = old_def[old_def.year!=year]

    new_def = pd.concat([old_def,defense])
    #new_def.drop(columns=['TEAM','PTS'],inplace=True)
    #new_off.drop(columns=['TEAM','PTS'],inplace=True)

    new_off.to_csv(offpath,index=False)
    new_def.to_csv(defpath,index=False)
update_team_masters(2024,offense,defense,ps=playoffs)

90
54
111
37
28
27
26
30
37
23
11
Index(['Player', 'PLAYER_ID', 'Team', 'GP', 'Poss', '% Time', 'PPP', 'Points',
       'FGM', 'FGA', 'FG%', 'aFG%', '%FT', '%TO', '%SF', 'AND ONEFREQ%',
       '%Score', 'Percentile', 'playtype', 'year'],
      dtype='object')
365
309
384
245
251
152
326
294
293
281
201


/tmp/ipykernel_2970/2895188510.py:246: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  old =pd.read_csv('playtype'+trail+'.csv')


Index(['Player', 'PLAYER_ID', 'Team', 'GP', 'Poss', '% Time', 'PPP', 'Points',
       'FGM', 'FGA', 'FG%', 'aFG%', '%FT', '%TO', '%SF', 'AND ONEFREQ%',
       '%Score', 'Percentile', 'playtype', 'year'],
      dtype='object')


In [2]:
#url_list = [url1]#
def get_multi(url_list,playoffs = False):
    if playoffs == True:
        p ='/playoffs/'
    else:
        p=''
    # get table per year, create corresponding directory if it doesn't exist
    
    for i in range(2023,2024):
        
        season = '&SeasonYear='+str(i)+'-'+str(i+1 - 2000)
        year_url = [url+season for url in url_list]
        frames = get_tables(year_url)

 
        path = str(i+1)+p+'/playtype/'
        output_dir = Path(path)
        output_dir.mkdir(parents=True, exist_ok=True)
        #terms = ['data/teampullup.csv','data/teamcatchshoot.csv','data/teamundersix.csv','data/teamiso.csv','data/teamtransition.csv']
        terms = ['handoff.csv','iso.csv','trans.csv','bh.csv','rollman.csv','postup.csv','spotup.csv',
                 'cut.csv','offscreen.csv','putback.csv','misc.csv']
        terms = [ path+t for t in terms]
        
        for i in range(len(terms)):
            df = frames[i]
            df.to_csv(terms[i],index = False)

In [3]:
#get_multi(url_list,playoffs = False)

In [4]:
#terms = ['data/teampullup.csv','data/teamcatchshoot.csv','data/teamundersix.csv','data/teamiso.csv','data/teamtransition.csv']
#frames = get_tables(url_list)
terms = ['playtype/handoff.csv','playtype/iso.csv','playtype/trans.csv','playtype/bh.csv','playtype/rollman.csv','playtype/postup.csv','playtype/spotup.csv',
         'playtype/cut.csv','playtype/offscreen.csv','playtype/putback.csv','playtype/misc.csv','playtype/drives.csv']


In [5]:
#df = pd.read_csv('2024/playtype/spotup.csv')
#df

In [6]:
def add_synergy():
    df = pd.read_csv('../synergy/full_data/playtype_p.csv')
    for i in range(2014,2024):
        path = str(i)+'/playoffs/synergy/'
        isExist = os.path.exists(path)
        if not isExist:

       # Create a new directory because it does not exist
           os.makedirs(path)
        year_df = df[df.year == i]
        print(year_df.head())
        year_df.to_csv(path+'playtype.csv')

In [29]:
def create_macro(data,play,playlist):
        num_col = ['% Time', 'PPP', 'Points', 'FGM', 'FGA','FG%', 'aFG%', '%FT', '%TO', '%SF', '%Score','Poss','GP']
        perc = [ 'PPP','FG%', 'aFG%', '%FT', '%TO', '%SF', '%Score']

        print(len(data.columns))
     
        data = data.loc[data.playtype.isin(playlist)]
        #data['playtype'] = play
        #data['GP']/=3
        group = data.groupby(['Player','Team','year'])
        s_list = []
        #print(p_df)
        for col in num_col:
            if col in perc:
                series =group.apply(w_avg, col, 'Poss')
            #poss = pd.Series([sum(df['Poss'])], index=['Poss'])
            #series = pd.concat([series,poss],keys =['series'])
            #print(series)
        #print(series)

            elif col =='GP':
                series = group.mean()[col]
                print(series)
         
            else:
                series = group.sum()[col]
            s_list.append(series)

        player_df = pd.concat(s_list)
        print(len(player_df.columns))
        player_df.columns = num_col
        #print(player_df)
        new_data = player_df.reset_index()
        new_data['playtype'] = play
        
        return new_data
def w_avg(df, values, weights):
    #print(values)
    #print(weights)

    d = df[values]
    w = df[weights]
    if values == 'Poss':

        return d.sum()
    
    return (d * w).sum() / w.sum()
df = pd.read_csv('playtype_p.csv')
print(df.columns)
#year = 2024
#df=df[df.year==year].reset_index(drop=True)
data_names = {'pr_ball':'on_ball','iso':'on_ball','pr_roll':'play_finish','post':'on_ball','hand_off':'motion'
               ,'oreb':'play_finish','cut':'play_finish','off_screen':'motion','spot':'play_finish','tran':'tran','misc':'misc'}
df['playtype'] = df['playtype'].map(data_names)
pstyle= df.groupby(['Player','Team','GP','PLAYER_ID','playtype','year']).sum()[['Poss','% Time','FGM','FGA','Points']].reset_index()
pstyle['PPP'] = pstyle['Points']/pstyle['Poss']

#pstyle.to_csv('play_style_p.csv',index=False)
pstyle

Index(['Position', 'Player', 'Team', 'GP', '% Time', 'Poss', 'Points', 'PPP',
       'FGm', 'FGM', 'FGA', 'FG%', 'aFG%', '%TO', '%FT', '%SF', '%Score',
       'year', 'playtype', 'And One Freq%', 'Percentile', 'FTFreq%',
       'TOVFreq%', 'SFFreq%', 'And OneFreq%', 'ScoreFreq%', 'PLAYER_ID',
       'AND ONEFREQ%'],
      dtype='object')


/tmp/ipykernel_2970/3812191222.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  old=pd.read_csv('playtype.csv')
/tmp/ipykernel_2970/3812191222.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  temp = pd.read_csv('playtype_backup.csv')


In [10]:
old['Player'].value_counts()

Player
Pascal Siakam       19
Kelly Olynyk        19
Bojan Bogdanovic    18
Buddy Hield         17
RJ Barrett          17
                    ..
P.J. Tucker          1
Jamaree Bouyea       1
Jeff Dowtin Jr.      1
Ashton Hagans        1
Marques Bolden       1
Name: count, Length: 435, dtype: int64

In [27]:
old=pd.read_csv('playtype.csv')
old = old[old.year==2024]
temp = pd.read_csv('playtype_backup.csv')
temp = temp[temp.year==2024]
id_map=dict(zip(old['Player'],old['PLAYER_ID']))
temp['PLAYER_ID'] = temp['Player'].map(id_map)
trans = pd.concat([old,temp])

trans = trans.drop_duplicates(subset=['Player','Team','GP','playtype','year'])
backed = pd.read_csv('playtype.csv')
backed = backed[backed.year<2024]
new_save = pd.concat([backed,trans])
new_save.to_csv('playtype.csv',index=False)
df = pd.read_csv('playtype.csv')
print(df.columns)
year = 2024
df=df[df.year==year].reset_index(drop=True)
data_names = {'pr_ball':'on_ball','iso':'on_ball','pr_roll':'play_finish','post':'on_ball','hand_off':'motion'
               ,'oreb':'play_finish','cut':'play_finish','off_screen':'motion','spot':'play_finish','tran':'tran','misc':'misc'}
df['playtype'] = df['playtype'].map(data_names)
pstyle= df.groupby(['Player','Team','GP','PLAYER_ID','playtype','year']).sum()[['Poss','% Time','FGM','FGA','Points']].reset_index()
pstyle['PPP'] = pstyle['Points']/pstyle['Poss']
oldstyle = pd.read_csv('play_style.csv')
oldstyle = oldstyle[oldstyle.year!=year]


newstyle = pd.concat([oldstyle,pstyle])
newstyle.to_csv('play_style.csv',index=False)

/tmp/ipykernel_2970/3032270722.py:1: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  old=pd.read_csv('playtype.csv')
/tmp/ipykernel_2970/3032270722.py:3: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  temp = pd.read_csv('playtype_backup.csv')
/tmp/ipykernel_2970/3032270722.py:10: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  backed = pd.read_csv('playtype.csv')
/tmp/ipykernel_2970/3032270722.py:14: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('playtype.csv')


Index(['Position', 'Player', 'Team', 'GP', '% Time', 'Poss', 'Points', 'PPP',
       'FGm', 'FGM', 'FGA', 'FG%', 'aFG%', '%TO', '%FT', '%SF', '%Score',
       'year', 'playtype', 'And One Freq%', 'Percentile', 'FTFreq%',
       'TOVFreq%', 'SFFreq%', 'And OneFreq%', 'ScoreFreq%', 'AND ONEFREQ%',
       'PLAYER_ID'],
      dtype='object')


,Position,Player,Team,GP,% Time,Poss,Points,PPP,FGm,FGM,...,playtype,And One Freq%,Percentile,FTFreq%,TOVFreq%,SFFreq%,And OneFreq%,ScoreFreq%,AND ONEFREQ%,PLAYER_ID
73380,NaN,Luka Doncic,Dallas Mavericks,70,23.3,511,555,1.086,NaN,184,...,iso,NaN,82.7,NaN,NaN,NaN,NaN,NaN,2.7,1629029.0
73381,NaN,Shai Gilgeous-Alexander,Oklahoma City Thunder,75,24.0,461,512,1.111,NaN,174,...,iso,NaN,86.4,NaN,NaN,NaN,NaN,NaN,3.5,1628983.0
73382,NaN,Jayson Tatum,Boston Celtics,73,25.9,464,483,1.041,NaN,166,...,iso,NaN,75.3,NaN,NaN,NaN,NaN,NaN,2.6,1628369.0
73383,NaN,Zion Williamson,New Orleans Pelicans,69,23.5,348,365,1.049,NaN,135,...,iso,NaN,77.0,NaN,NaN,NaN,NaN,NaN,6.9,1629627.0
73384,NaN,Anthony Edwards,Minnesota Timberwolves,79,17.5,352,326,0.926,NaN,115,...,iso,NaN,50.2,NaN,NaN,NaN,NaN,NaN,2.3,1630162.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76090,NaN,Sidy Cissoko,San Antonio Spurs,12,30.2,13,5,0.385,NaN,2,...,spot,NaN,0.8,NaN,NaN,NaN,NaN,NaN,0.0,1631321.0
76091,NaN,Trevelin Queen,Orlando Magic,14,30.0,15,5,0.333,NaN,2,...,spot,NaN,0.3,NaN,NaN,NaN,NaN,NaN,6.7,1630243.0
76092,NaN,Mason Plumlee,LA Clippers,46,4.3,10,4,0.400,NaN,0,...,spot,NaN,1.1,NaN,NaN,NaN,NaN,NaN,0.0,203486.0
76093,NaN,Yuta Watanabe,Memphis Grizzlies,5,52.2,12,4,0.333,NaN,2,...,spot,NaN,0.3,NaN,NaN,NaN,NaN,NaN,0.0,1629139.0


,Position,Player,Team,GP,% Time,Poss,Points,PPP,FGm,FGM,...,playtype,And One Freq%,Percentile,FTFreq%,TOVFreq%,SFFreq%,And OneFreq%,ScoreFreq%,AND ONEFREQ%,PLAYER_ID
73380,NaN,Giannis Antetokounmpo,Milwaukee Bucks,72,23.5,454,558,1.230,NaN,198,...,tran,NaN,71.2,NaN,NaN,NaN,NaN,NaN,7.9,203507.0
73381,NaN,Shai Gilgeous-Alexander,Oklahoma City Thunder,75,22.0,422,511,1.210,NaN,184,...,tran,NaN,68.7,NaN,NaN,NaN,NaN,NaN,3.3,1628983.0
73382,NaN,Jaylen Brown,Boston Celtics,69,27.3,414,487,1.180,NaN,185,...,tran,NaN,59.9,NaN,NaN,NaN,NaN,NaN,3.4,1627759.0
73383,NaN,LeBron James,Los Angeles Lakers,70,22.8,375,477,1.270,NaN,173,...,tran,NaN,79.7,NaN,NaN,NaN,NaN,NaN,6.9,2544.0
73384,NaN,Anthony Edwards,Minnesota Timberwolves,79,18.2,366,456,1.250,NaN,160,...,tran,NaN,74.2,NaN,NaN,NaN,NaN,NaN,5.5,1630162.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76066,NaN,Kobe Bufkin,Atlanta Hawks,17,28.6,30,19,0.633,NaN,8,...,spot,NaN,4.2,NaN,NaN,NaN,NaN,NaN,3.3,1641723.0
76068,NaN,Trent Forrest,Atlanta Hawks,38,16.7,20,18,0.900,NaN,8,...,spot,NaN,23.7,NaN,NaN,NaN,NaN,NaN,0.0,1630235.0
76077,NaN,Lamar Stevens,Boston Celtics,19,19.0,11,13,1.182,NaN,4,...,spot,NaN,85.6,NaN,NaN,NaN,NaN,NaN,0.0,1630205.0
76082,NaN,Kevon Looney,Golden State Warriors,74,3.8,12,9,0.750,NaN,4,...,spot,NaN,10.1,NaN,NaN,NaN,NaN,NaN,0.0,1626172.0


,Position,Player,Team,GP,% Time,Poss,Points,PPP,FGm,FGM,...,playtype,And One Freq%,Percentile,FTFreq%,TOVFreq%,SFFreq%,And OneFreq%,ScoreFreq%,AND ONEFREQ%,PLAYER_ID
73382,NaN,Jaylen Brown,Boston Celtics,69,27.3,414,487,1.180,NaN,185,...,tran,NaN,59.9,NaN,NaN,NaN,NaN,NaN,3.4,1627759.0
73802,NaN,Jaylen Brown,Boston Celtics,69,16.3,247,231,0.940,NaN,94,...,pr_ball,NaN,71.8,NaN,NaN,NaN,NaN,NaN,1.6,1627759.0
74464,NaN,Jaylen Brown,Boston Celtics,69,10.7,162,157,0.970,NaN,59,...,iso,NaN,59.8,NaN,NaN,NaN,NaN,NaN,2.5,1627759.0
74958,NaN,Jaylen Brown,Boston Celtics,69,9.1,138,145,1.050,NaN,61,...,post,NaN,67.5,NaN,NaN,NaN,NaN,NaN,3.6,1627759.0
75784,NaN,Jaylen Brown,Boston Celtics,69,4.3,65,81,1.250,NaN,36,...,cut,NaN,34.9,NaN,NaN,NaN,NaN,NaN,3.1,1627759.0
76072,NaN,Jaylen Brown,Boston Celtics,69,4.3,66,52,0.790,NaN,20,...,hand_off,NaN,28.3,NaN,NaN,NaN,NaN,NaN,4.5,1627759.0
76305,NaN,Jaylen Brown,Boston Celtics,69,4.5,68,75,1.100,NaN,29,...,off_screen,NaN,67.0,NaN,NaN,NaN,NaN,NaN,2.9,1627759.0
75533,NaN,Jaylen Brown,Boston Celtics,69,2.9,44,51,1.159,NaN,23,...,oreb,NaN,57.5,NaN,NaN,NaN,NaN,NaN,11.4,1627759.0
75809,NaN,Jaylen Brown,Boston Celtics,69,16.6,252,258,1.024,NaN,91,...,spot,NaN,45.6,NaN,NaN,NaN,NaN,NaN,2.4,1627759.0


In [25]:
len(old)

3101